<div id="container" style="position:relative;">
<div style="float:left"><h1> Recommender Systems </h1></div>
<div style="position:relative; float:right"><img style="height:65px" src ="https://drive.google.com/uc?export=view&id=1EnB0x-fdqMp6I5iMoEBBEuxB_s7AmE2k" />
</div>
</div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In this lecture we will focus on one of the most widely used data science applications, recommender systems. At a high level, these systems aim to predict how a user feels about various items. We will focus on specific applications of these predictions, develop our own basic systems and see how to evaluate them.

### What are Recommender Systems

Generally a recommender system provides a user with a list of options it thinks they would like to try. You probably see these systems every day on websites like Amazon or Spotify. These companies want you to continually use their service and make purchases from them, their income is highly dependent on providing recommendations you follow up on.

<img src = "https://drive.google.com/uc?export=view&id=1TxpAHfLkp4PkVerZ0dRF3bySthWYQeKD" width=80%>

At its core, a recommender system contains information about many users and their ratings of certain items, and it tries to predict what kind of rating a user will give an item they have never seen before.

### Movie Rating System

We're going to use a dataset of movies from the internet movie database, that is our system will recommend movies to a user. 

Let's load up some basic data to work with:

In [ ]:
movie_df = pd.read_csv('data/movies_metadata.csv')
movie_df.drop_duplicates(subset=['title'], inplace=True)
movie_df = movie_df.reset_index().drop('index', axis=1)
movie_df.head()

In [ ]:
movie_df.info()

In [ ]:
movie_df = movie_df[['id', 'title', 'overview', 'vote_average', 'vote_count']]
movie_df.head()

In [ ]:
movie_df.shape

### A User Independent System

We'll start with building recommendations for the case where we have no knowledge about a user. In this case, we probably would find the most popular items, and recommend them to our user.

The simplest system would just sort the items based on their average score and present a list in that order:

In [ ]:
top_rated = movie_df.sort_values(by=['vote_average'], ascending=False)
top_rated['title'].head(10)

Those results seem a bit interesting... Let's look at the top few items in more detail:

In [ ]:
top_rated.head(10)

There's a big issue with this system, the top ranked items have a perfect average score but only one review. Let's look at a movie which is generally considered highly rated:

In [ ]:
top_rated[top_rated['title'] == "Citizen Kane"]

Citizen Kane, which many consider the greatest movie ever made, only has a score of $8.0$ but it has over $1200$ reviews. The problem is a more esoteric movie which is only known of by a very narrow audience is likely to have a few extremely positive reviews. But this is probably something we wouldn't want to push to the top of a movie recommendation list!

One way to deal with items with few reviews is to simply ignore them. This is known as thresholding, we set some threshold for required reviews, we simply ignore items with too few reviews. If we were to implement this with our data set:

In [ ]:
threshold = 1000
movies_with_many_reviews_df = movie_df[movie_df['vote_count'] >= threshold]

top_rated_v2 = movies_with_many_reviews_df.sort_values(by=['vote_average'], ascending=False)
top_rated_v2.head(10)

We see the recommendations have significantly changed. While this is a nice fix there are two major issues:

1. We selected the threshold in an arbitrary fashion, it seems like a reasonable number, but we kind of just made it up.
2. If an item must meet a threshold to appear in a recommended list but it never appears in a list not many people will watch it. That is it will have a hard time getting the reviews it needs. 

The first issue can be difficult to deal with, but we can perform user studies and see what is a reasonable threshold. For the second issue one possible fix is to occasionally ignore the threshold requirements for new items, this allows them to build up some initial reviews.

#### User-independent System Review

When we have no information about a user or their preferences, we can just show them the most popular movies given some review threshold. For new movies, we can sometimes show them to users and ignore the thresholding.

---------

### Content Based Recommendations

What if we know something about our user? For example, we know they like Iron Man movies. Then we can employ content-based recommendations.

These recommendation engines are built around the idea if a user likes some item (or a particular basket of items) then they will like similar items based on the item content/description. If I watched the Avengers, then I probably would want to watch other superhero-themed movies. If we look at a few movies along with their descriptions:

In [ ]:
df_descriptions = movie_df[['title', 'overview']]
df_descriptions.head(10)

What does it mean for two items to be similar from a content perspective? 

Well first, we need to put the content into a format a model can understand. Right now we only have movie descriptions and we can't really feed this straight into some model. Instead we will use the Term Frequency Inverse Document Frequency measure (TF-IDF).


The TF-IDF is made of two measures, term frequency (TF) and inverse document frequency (IDF). 

For a term (a word or phrase), $t$, and a document ,$d$, the term frequency $\text{TF}(t,d)$, measures how common the term is in the document. 

The inverse document frequency of a term $t$, $\text{IDF}(t)$, is the inverse of the number of documents a term $t$ appears in.

The intuition is that when a term appears in many documents, it's not very relevant. However, if it only appears in a few documents, it's very relevant to identifying those documents.

For a complete description see the [Wikipedia](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) page but what you need to know is: 

- The more common a term is across all documents the more $\text{IDF}(t)$ shrinks, 
- The more common a term is within one document the more $\text{TF}(t,d)$ grows. 
- The term frequency inverse document frequency is simply the product of these two: 
    $$\text{TF-IDF}(t,d) = \text{TF}(t,d)\times  \text{IDF}(t)$$

Now we can transform our data using the TF-IDF measure. Each movie is a document, or more precisely each description is a document, and for now let's keep it simple, terms will just be individual words. We can use built in python functionality to make our life easy:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words = "english", min_df=2)
movie_df['overview'] = movie_df['overview'].fillna("")

TF_IDF_matrix = vectorizer.fit_transform(movie_df['overview'])

In [ ]:
# 42,728 movies by 36,504 tokens in the descriptions =~ 1.5B elements!
TF_IDF_matrix.shape

The matrix returned is quite large, each row is a document and each column is a term. Entry ${(i,j)}$ is just the TF-IDF$(i,j)$.

Now that we have our data in a numeric format how can we measure the similarity between two documents. Each document is a row in our sparse matrix:

In [ ]:
TF_IDF_matrix[(movie_df['title'] == 'Citizen Kane').values].todense().squeeze()

We see these are just numeric arrays, that is vectors. There are several common ways to compare two vectors $a$ and $b$, probably the most common in recommender systems is the cosine similarity:

$$\frac{a \cdot b}{||a||\cdot ||b||}$$

The closer two vectors (or documents) are, the higher this measure. We can calculate the similarity using cosine similiarity using sklearn's cosine similarity function:

In [ ]:
movie_df[movie_df['title'].str.contains('Captain America', na=False)]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

movie_1 = TF_IDF_matrix[(movie_df['title'] == 'Captain America: The First Avenger').values,]
movie_2 = TF_IDF_matrix[(movie_df['title'] == 'Captain America: The Winter Soldier').values,]

print("Similarity:", cosine_similarity(movie_1, movie_2)) # Notice the result is a 2D 1X1 array, so to grab
                                                          # the number we will need to index

Not only can we use the `sklearn.metrics.pairwise.cosine_similiarity` function to compute that between two different vectors, we can pass the entire tf-idf matrix into the function as a single argument and it will compute the similarity between each column and every other column, giving back a square matrix, where the entry at $(i, j)$ is the similarity between movie $i$ and $j$ (like a correlation matrix for features).

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(TF_IDF_matrix, dense_output=False)

In [ ]:
# Check the shape
# rows and columns should be equal, and the number of movies we started with (rows)
similarities.shape

Now that we can directly compare two movies and we can make recommendations of the form: if you like movie $a$ then you will also like movies $b$, $c$, $d$, $etc$. 

We can do this just picking a candidate film and taking its column in the similarity matrix, and then finding those rows where the similarities are highest:

In [ ]:
# Test with a sample movie
movie_df[movie_df['title'] == 'Captain America: The Winter Soldier']

In [ ]:
# Get the column based upon the index
movie_index = movie_df[movie_df['title'] == 'Captain America: The Winter Soldier'].index

# Create a dataframe with the movie titles
sim_df = pd.DataFrame({'movie':movie_df['title'], 
                       'similarity': np.array(similarities[movie_index, :].todense()).squeeze()})

In [ ]:
# Return the top 10 most similar movies
sim_df.sort_values(by='similarity', ascending=False).head(10)

Now let's now implement this using a simple function, since we must first find the corresponding column in the similarity matrix. We'll also add a parameter for a vote threshold, to restrict our results to above a certain threshold of votes, so that we can only return more popular films if we'd like:

In [ ]:
def content_recommender(title, movies, similarities, vote_threshold=10) :
    
    # Get the movie by the title
    movie_index = movies[movies['title'] == title].index
    
    # Create a dataframe with the movie titles
    sim_df = pd.DataFrame(
        {'movie': movies['title'], 
         'similarity': np.array(similarities[movie_index, :].todense()).squeeze(),
         'vote_count': movies['vote_count']
        })
    
    # Get the top 10 movies with > 10 votes
    top_movies = sim_df[sim_df['vote_count'] > vote_threshold].sort_values(by='similarity', ascending=False).head(10)
    
    return top_movies

In [ ]:
# Test the recommender
similar_movies = content_recommender("Thor", movie_df, similarities, vote_threshold=1000)
similar_movies.head(10)

### Content Based System Review

These systems try and recommend new items (in our case movies) based on the idea that if you like a certain movie, you will like movies that are similar in their description. We could also include more information than just the vectorized description text by adding other features, for example, whether films were of a particular genre or had a particular director or actor.

This is a more advanced recommendation system than the user independent one, and certainly gives better recommendations. However we also need to be mindful about sometimes returning some variety in our results to make sure users don't get bored with our recommendations.

------

### Collaborative Based Recommendations

Collaborative filtering also relies on similarity between items, as well as similarity between users. However, it define similarity slightly differently.

The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion/taste as a person B on an issue/item, A is more likely to have B's opinion on a different issue/item than that of a randomly chosen person.

Collaborative methods for recommender systems are methods that are based solely on the past interactions recorded between users and items in order to produce new recommendations. These interactions are stored in the so-called “user-item interactions matrix”.



<div>
<img src="https://miro.medium.com/v2/resize:fit:1000/1*m_Z6Da5FZ62KN2yH-x_GOQ@2x.png" width="1000"/>
</div>


Unlike content-based systems, a collaborative system looks at an item as a collection of ratings. Every item has ratings by some users, and if two items get very similar ratings from users, the items themselves are similar (notice this system is not at all aware of the items' content).



Similarly, we can define users to be similar if they rate items similarly.


To actually do collaborative filtering we'll need some more data, now we'll user data with movie reviews from various users:

In [ ]:
columns = ['user', 'movie', 'score', 'timestamp']
df = pd.read_csv('data/u.data', sep='\t', names=columns).drop('timestamp',axis=1)
print(df.shape)
df.head()

Notice we only have the review for each user and movie combination. In essence, we know user 196 gave movie 242 a review of 3, but we know nothing of the movie content or the user preference.

The first thing we need to do is turn the dataframe we have into a matrix $R$, where each entry in $R$, $R_{um}$ is the rating user $u$ gave item $m$. This is referred to as the _utility matrix_. 

In our case the items, are movies. Let's get our user-item utility matrix $R$:

In [ ]:
users = df['user'].unique()
movies = df['movie'].unique()

num_users = len(users)
num_movies = len(movies)
print('num unique users:', len(users))
print('num unique movies:', len(movies))
           
# np.nan means they user has not reviewed the movie
R = np.full((num_users, num_movies), np.nan)

#Build the user-item matrix
for row in df.itertuples(): # same as zip(df.index, df["user"], df["movie"], df["score"])
    user = row[1]
    movie = row[2]
    rating = row[3]
    R[user-1, movie-1] = rating
    
R_df = pd.DataFrame(data=R, index=range(1, num_users+1), columns=range(1, num_movies+1))

In [ ]:
list(df.itertuples())[0] # Pandas special named tuple

In [ ]:
R_df

In this matrix, the rows are the different users (so we have 943 different users) and the columns are the movies (so we have 1,682 different movies). 

As expected, most of the entries are empty since not every user has watched every film.

### User-Item Filtering

Our first collaborative filtering model will be **_user-item filtering_**. User-item filtering works on the logic users with similar preferences on some item have similar preferences over other items. If we wanted to know what $R_{ij}$ is (what we think user $i$ would give to item $j$) we would find users similar to $i$ and determine what rank they gave $j$, we can use this as an estimate for $R_{ij}$.

### Similar Users

What makes two users similar? The simplest way to measure if user $a$ and $b$ are similar is to look at the items both of them reviewed. For example consider the following two users:

In [ ]:
R_df.loc[1, :]

In [ ]:
R_df.loc[16, :]

We can compare them directly on the first movie since they both reviewed each of them. But we can't compare them on the second and third movie since only the first user reviewed both of those.


We can find all of the movies that they both reviewed with:

In [ ]:
movies_1st_user_rated = ~R_df.loc[1, :].isna()
movies_2nd_user_rated = ~R_df.loc[16, :].isna()

movies_both_users_rated = movies_1st_user_rated & movies_2nd_user_rated
print(movies_both_users_rated.sum())

In [ ]:
print("Reviewers' scores:")
R_df.loc[[1, 16], movies_both_users_rated]

This subset of items forms a vector over the same dimensions (items) for each user, thus we can take some measure of vector similarity to find user similarity. Let's again use the cosine similarity to find how close two users are:

In [ ]:
print("Similarity:", cosine_similarity(R_df.loc[1, movies_both_users_rated].values.reshape(1,-1), 
                                       R_df.loc[16, movies_both_users_rated].values.reshape(1,-1)))

### Making Predictions

Now that we have a way of directly comparing users we can find how similar user $u_i$ is to every other user who reviewed item $m$ (if we implement this for user 16 and item 1) we get:

In [ ]:
def find_user_similarity(user_1, user_2, R_df):
    
    # Define the mask which finds all movies they rated together
    movies_1st_user_rated = ~R_df.loc[user_1, :].isna()
    movies_2nd_user_rated = ~R_df.loc[user_2, :].isna()

    movies_both_users_rated = movies_1st_user_rated & movies_2nd_user_rated

    # Sum boolean to get the counts
    number_of_movies_rated_together = movies_both_users_rated.sum()
        
    # Find the ratings of both users for movies they both watched
    ratings_of_user1 = R_df.loc[user_1, movies_both_users_rated].values.reshape(1, -1)
    ratings_of_user2 = R_df.loc[user_2, movies_both_users_rated].values.reshape(1, -1)
    
    # Finally, calculate the similarity between them
    similarity = cosine_similarity(ratings_of_user1, ratings_of_user2)[0][0]
    
    return similarity, number_of_movies_rated_together

In [ ]:
current_user = 16 # we will only do this to user 16 for demonstration purposes
current_movie = 2
similarities_to_user_16 = []
ratings_given_to_movie_2 = []

# Find only the users who rated movie 2 (rows)
R_df2 = R_df[~R_df.iloc[:, 1].isna()].copy()

for other_user in R_df2.index:
    
    similarity, number_of_movies_rated_together = find_user_similarity(current_user, other_user, R_df)
    similarities_to_user_16.append(similarity)
    ratings_given_to_movie_2.append(R_df.loc[other_user, current_movie])
            
# Finally, let's turn these into numpy arrays so life is easier
similarities_to_user_16 = np.array(similarities_to_user_16)
ratings_given_to_movie_2 = np.array(ratings_given_to_movie_2)

Now we can calculate the expected rating user 16 will give movie 2. We do so by a weighted average; We grab the score user $v$ gave movie 2, and multiply it by the similarity between user 16 and user $i$. Finally we divide by the sum of similarities. We do this for all $C$ users we have found that rated movies user 16 has watched.

This can all be expressed as:

$$\hat{R_{16, 2}} = \frac{\sum_{v=1}^C R_{v, 2} \cdot \text{user_similarity}(16,v)}{\sum_{v=1}^C\text{user_similarity}(16, v)}$$

for our particular case, or for the general case of user $u$ and movie $i$

$$\hat{R_{um}} = \frac{\sum_{v=1}^C R_{vm} \cdot \text{user_similarity}(u, v)}{\sum_{v=1}^C \text{user_similarity}(u, v)}$$

In [ ]:
predicted_rating = np.dot(ratings_given_to_movie_2, similarities_to_user_16)/np.sum(similarities_to_user_16)

print(f'Predicted rating for movie 2 by user 16 is {round(predicted_rating, 2)}')

Now to find the overall recommendation list for user 16 we need to calculate this score for every item they haven't rated yet then sort that list, quite the task!

------------------------



### Item-Item Filtering

Our second collaborative filter model will be item-item filtering. In user-item filtering, we said if two users are similar, and one of them rated item $i$, the other user's rating of item $i$ will be similar.

In **_item-item filtering_**, we say that if two items are similar, and a user ranked one of those items, that user's rating of the other item will be similar.

In essence the logic is very similar, except instead of judging user similarity, we will look at item similarity.

### Similar Items

What makes two items similar? As with measuring if users are similar there are many ways of measuring item similarity. For our simple method to find the similarity of two items, $x$ and $y$, we look at every user who reviewed both items. For example let us consider the items in columns 15 and 18. 

In [ ]:
users_who_ranked_item_15 = ~R_df.loc[:, 15].isna()
users_who_ranked_item_18 = ~R_df.loc[:, 18].isna()

users_who_ranked_both_items = users_who_ranked_item_15 & users_who_ranked_item_18
print(users_who_ranked_both_items.sum())

In [ ]:
print("Reviewers' scores:")
R_df.loc[users_who_ranked_both_items, [15, 18]]

Now we can judge how similar the ratings between items 15 and 18 are.

In [ ]:
R_df.loc[users_who_ranked_both_items, 18].values.reshape(1,-1)[0]

In [ ]:
print("Similarity:", cosine_similarity(R_df.loc[users_who_ranked_both_items, 15].values.reshape(1,-1), 
                                       R_df.loc[users_who_ranked_both_items, 18].values.reshape(1,-1))[0][0])

### Making Predictions 

Now that we can measure how similar two items are we can use this information to make a prediction. The idea behind item-item filtering is the review user $u$ gives to item $i$ ($R_{um}$) should be similar to the review $u$ gives to a similar item $n$ ($R_{un}$), if items $m$ and $n$ are similar. 

We can start with an idea similar to what we did with user-item predictions. We'll define a set of items similar to $m$ that user $u$ reviewed. We can take the average rating for each item in this list and use it as the value for $R_{um}$. This time $C$ is our number of total movies:

$$\hat{R_{um}} = \frac{\sum_{n=1}^C R_{un} \cdot \text{item_similarity}(m, n)}{\sum_{n=1}^C \text{item_similarity(m, n)}}$$

#### Exercise 1

1. Finish implementing the item-item similarity function, the function should return -1, 0 if no users rated both items (look at the previous similarity function), and the similarity of the two items and number of reviewers otherwise.

    Use this function to predict the rating user 16 would give movie 2. How does this compare with the previous result (user-item)?

#### Solutions

In [ ]:
def find_item_similarity(item_1, item_2, R_df):
    
    # Define the mask which finds users who rated both items
    users_who_reviewed_1st_item = R_df.loc[:, item_1].notna()
    users_who_reviewed_2nd_item = R_df.loc[:, item_2].notna()

    users_who_reviewed_both_items = users_who_reviewed_1st_item & users_who_reviewed_2nd_item
    
    # Find how many users rated both items
    number_of_users = users_who_reviewed_both_items.sum()
    
    # Find the ratings of both users for movies they both watched
    ratings_of_item1 = R_df.loc[users_who_reviewed_both_items, item_1].values.reshape(1, -1)
    ratings_of_item2 = R_df.loc[users_who_reviewed_both_items, item_2].values.reshape(1, -1)
    
    # Finally, calculate the similarity between them
    similarity = cosine_similarity(ratings_of_item1, ratings_of_item2)[0][0]
    
    return similarity, number_of_users

In [ ]:
current_user = 16 # we will only do this to user 16 for demonstration purposes
current_movie = 2
similarities_to_movie_2 = []
ratings_given_by_user_16 = []

# Find only the items rated by user 16 (columns)
R_df3 = R_df.loc[:,~R_df.iloc[15,:].isna()].copy()

for other_movie in R_df3.columns:
    
    similarity, number_of_user_rated = find_item_similarity(current_movie, 
                                                                other_movie, 
                                                                R_df)
    # Append similarity and ratings
    similarities_to_movie_2.append(similarity)
    ratings_given_by_user_16.append(R_df.loc[current_user, other_movie])
            
# Finally, let's turn these into numpy arrays so life is easier
similarities_to_movie_2 = np.array(similarities_to_movie_2)
ratings_given_by_user_16 = np.array(ratings_given_by_user_16)

In [ ]:
predicted_rating = np.dot(ratings_given_by_user_16, similarities_to_movie_2)/np.sum(similarities_to_movie_2)

print(f'Predicted rating for movie 2 by user 16 is {round(predicted_rating, 2)}')

### Memory Based Filtering

These two methods are instances of what are generally known as memory based filtering. While there is no hard definition, it is generally understood to mean methods which work by using the entire set of user and item reviews  calculating the relevant information from this matrix.

### Collaborative  System Review

Collaborative filtering is a powerful family of methods which leverage the decisions made by groups of users to make predictions. These models form the basis of various real world system and they tend to perform well in practice.

That being said, there are drawbacks to collaborative systems. One of the biggest issues is the size and sparsity of the data. Consider the user-item matrix in our previous example. This matrix is huge, and it's from a pretty small example. Imagine the Amazon user-item data set: there are over three-hundred million active users and almost half a billion products. There's no way we could run our algorithms on it very frequently (if ever).

-------

### Matrix Factorization Methods and Latent Features (Model Based CF)

The last method we are going to discuss is the matrix factorization method. Consider the imaginary case where we have only four movies and three users, and we know all the ratings. That is, we have the following utility matrix:

$$R = \begin{bmatrix} 4 & 1 & 1 & 5\\ 5 & 5 & 2 & 1\\ 3 & 5 & 4 & 3\end{bmatrix}$$


We can imagine that we can decompose this matrix into two other matrices, $U$ (for users) and $M$ (for movies), where 

$$U \cdot M = R$$

We can also imagine that we need to define the shape of $U$ and $M$. Let's say we want $U$ to be a $3\times 3$ matrix and $M$ to be a $3\times 4$ matrix.


Through a bit of math that we won't discuss now, we can find two matrices which satisfy our demands. Their approximate values will be:


$$U = \begin{bmatrix} 0.029 & 2.898 & 0.46\\ 2.33 & 0.61 & 0.0\\ 0.561 & 0.0 &1.87\end{bmatrix}$$

and 


$$M = \begin{bmatrix} 1.78 & 2.13 & 0.75 & 0.0\\ 1.36 & 0.0 & 0.37 & 1.63\\ 0.0 & 2.03 & 1.91 & 0.53\end{bmatrix}$$


We've now decomposed the rating matrix into a matrix that tells us something about users and movies. The columns of the $U$ matrix and rows of the $M$ matrix tell us something about each user and movie, it tells us about some _latent_ (hidden) variables for each user and movie.

To find out what the latent variables are, we need to know more about items being rated.

Imagine our rating matrix was actually labeled like so:

|User  | Finding Nemo |  Thor  | Thor: Ragnarok |  Finding Dory  |
|------|--------------|--------|----------------|----------------|
|Bob   |   4          | 1      |   1            | 5              |
|Sally |   5          | 5      |   2            | 1              |
|Shila |   3          | 5      |   4            | 3              |

This means our user matrix is as follows:

|      | Latent Variable1 |  Latent Variable2  | Latent Variable3|
|------|------------------|--------------------|-----------------|
|Bob   |   0.29           |         2.89       |   0.46          |
|Sally |   2.33           | 0.61               |   0.0           |
|Shila |   0.56           | 0.0                |   1.87          |


and our movie matrix is as follows:

|                 | Finding Nemo |  Thor  | Thor: Ragnarok |  Finding Dory  |
|-----------------|--------------|--------|----------------|----------------|
|Latent Variable1 |   1.78       | 2.13   |   0.75         | 0.0            |
|Latent Variable2 |   1.36       | 0.0    |   0.37         | 1.63           |
|Latent Variable3 |   0.0        | 2.03   |   1.91         | 0.53           |


Latent variable 1 seems to be concerned with original movies vs sequels (Finding Nemo vs Finding Dory, and Thor vs Thor: Ragnarok). Latent variable 2 seems to be concerned with CG movies (or movies about marine life, we're not sure). Finally, latent variable 3 seems to be most active with Chris Hemsworth movies (or Marvel movies).


Looking at our users, we can see that Bob is very high in his score of latent variable 2, he likes CG movies about marine life. Sally, on the other hand, is mostly concerned with originals vs sequels. Finally, Shila is interested in Marvel movies or Chris Hemsworth movies. Now we know something about each user and each movie that wasn't explicit in the data provided, we had to infer it by inspecting the movies and users.

To be clear, we could programmatically find the values of the latent variables, but we had to perform manual inspection to understand what the variables represent.

### Funk Singular Value Decomposition (FunkSVD)

Actually, we can't use traditional matrix factorization techniques on our rating matrix. Recall what our rating matrix looks like

In [ ]:
R_df

It's mostly filled with NaN values, and we can't impute them because that will throw off the mathematical methods. **So what do we do?**

Well, we are going to use a method called [FunkSVD](https://sifter.org/~simon/journal/20061211.html) (which stands for Funk Singular Value Decomposition, invented by Simon Funk - won the Netflix competition). This method treats this as an optimization problem.

We have our matrix with known values, and we want to find $x$ latent variables. We will try and find two matrices; $U$ and $M$ which get as close as possible to the values we DO know.

This method is implemented in the `surprise` package ([more documentation here](https://surprise.readthedocs.io/en/stable/matrix_factorization.html)).

In [ ]:
from surprise import Dataset
from surprise.reader import Reader
from surprise.prediction_algorithms.matrix_factorization import SVD as FunkSVD

Because of the way FunkSVD in `surprise` works, it assigns each user with a value starting at 0. It also does so by the order it has seen users (it doesn't assume user will be recorded as numbers, we can have unique string IDs there and the algorithm will work just the same).

Let's sort the dataframe so we have a slightly easier time making sense of the conversion FunkSVD does

In [ ]:
original_df = df.sort_values(by=['user', 'movie'])
original_df.head()

We need to load our dataframe as a special `Dataset` object from `surprise`

In [ ]:
my_dataset = Dataset.load_from_df(original_df, Reader(rating_scale=(1, 5)))
my_train_dataset = my_dataset.build_full_trainset()

In [ ]:
my_train_dataset

Now all we do is initialize the algorithm, specify the number of latent variables and iterations we'd like to use, and then let the algorithm run.

In [ ]:
my_algorithm = FunkSVD(n_factors=10, 
                       n_epochs=100, 
                       lr_all=0.1,    # Learning rate for each epoch
                       biased=False,  # This forces the algorithm to store all latent information in the matrices
                       verbose=0)

my_algorithm.fit(my_train_dataset)

The user matrix is stored under the `SVD.pu` attribute

In [ ]:
U = my_algorithm.pu
U.shape

and the movie matrix is stored under the `SVD,qi` attribute, notice it is transposed

In [ ]:
M = my_algorithm.qi.T
M.shape

If we had more information about each item, we could begin exploring what the latent factors are by picking a movie

In [ ]:
first_movie = M[:, 0]

plt.figure(figsize=(15, 7))
plt.barh([f'Latent Var{i}' for i in range(1,len(first_movie)+1)], first_movie)
plt.title("Latent Variable Composition of Movie #1")
plt.ylabel("Latent Variable")
plt.xlabel("Value")
plt.show()

However, what we _can_ do is now make new predictions. Let's explore our original movie rating data:

In [ ]:
R_df.iloc[:5, :5]

We know the rating user 1 gave movie 2 (it's 3.0), so let's use this to demonstrate how we calculate ratings using these latent factors.

First, we grab the user profile

In [ ]:
inner_user_id = my_train_dataset.to_inner_uid(1) # find the inner representation of user 1
user_profile = U[inner_user_id]
user_profile

In [ ]:
plt.figure(figsize=(15, 7))
plt.barh([f'Latent Var{i}' for i in range(1,len(first_movie)+1)], user_profile)
plt.title("Latent Variable Profile of User #1")
plt.ylabel("Latent Variable")
plt.xlabel("Value")
plt.show()

Then, we grab the movie profile

In [ ]:
inner_movie_id = my_train_dataset.to_inner_iid(2) # find the inner representation of item 1
movie_profile = M[:, inner_movie_id]
movie_profile

In [ ]:
plt.figure(figsize=(15, 7))
plt.barh([f'Latent Var{i}' for i in range(1,len(first_movie)+1)], movie_profile)
plt.title("Latent Variable Profile of Movie #2")
plt.ylabel("Latent Variable")
plt.xlabel("Value")
plt.show()

Our expected rating of this movie by this user is the dot product of these two profiles:

In [ ]:
expected_rating = np.dot(user_profile, movie_profile)
expected_rating

In reality, our ratings will get better and better as we use more latent variables. However, too many latent variables and we might be overfitting.

#### Matrix Factorization Review

The matrix factorization methods treat the rating prediction problem as an optimization problem. They try to optimize two matrices which combine to form the observed ratings. 

An added benefit the provide is creating profiles of user and movies that can be inspect to find underlying trends. Like all optimization methods we saw, these methods come with a set of hyperparameters to adjust:

* **epochs**: Number of iterations the algorithm runs for. 
* **learning rate**: The speed at which the algorithm learns. Larger values give faster learning, but smaller values give more accurate learning. In `surprise`, you can adjust a global learning rate, or a learning rate per matrix.
* **number of factors**: The number of latent factors the algorithm attempts to learn. More factors can give better results, but can also lead to overfitting

### Recommender Systems Evaluation

How do we evaluate the quality of our predictions and recommendations? Is there a standard metric used in practice? First we need to settle on what are we evaluating:
- is it the score we fill into the user-item matrix?
- is it the actual list of recommended items?
- is it the utility of the recommended items (did it prompt user engagement)?

To compare true and predicted scores, we often use the *root mean square error* (RMSE):

$$\sqrt{\frac{1}{n}\sum_{ij}(R_{ij}-\hat R_{ij})^2}$$

Here, $n$ stands for the number of scores and $\hat R_{ij}$ is our prediction for the actual score $R_{ij}$. This seems like a reasonable measure, it's basically a regression measure, but is it any good? There are a few problems with it:

1. **It can only measure the accuracy on user-item pairs we have data for.** What if our system makes a recommendation for a previously unseen item, which it really should be doing, how do we evaluate this? Are the reviews on seen items correlated with the reviews on unseen items?
2. **Does an increase in performance (lower RMSE) imply an increase in revenue?** The end goal of the system is to increase user interaction and revenue, but do better score predictions mean more money? Intuitively it seems like suggesting products a user likes should increase money but there is no hard evidence backing this up.
3. **Different users rank movies on different, often personal scales**: using the same 1-10 scale, some people never give too harsh scores while others submit only highly polarized reviews. This difference often hinders regression metrics to capture the real performance of a recommender system. This can be addressed though by scaling each user's recommendation before modeling.

The `surprise` package implements RMSE and a few other evaluation metrics:

In [ ]:
from surprise import accuracy
from surprise.model_selection import train_test_split

# The surprise package doesn't allow you to test on the trainset we built
my_train_dataset, my_test_dataset = train_test_split(my_dataset, test_size=0.5)

predictions = my_algorithm.test(my_test_dataset)

In [ ]:
# RMSE
RMSE = accuracy.rmse(predictions, verbose=False)
print(RMSE)

Similar to RMSE, it is quite common to evaluate the *mean squared error* (MSE)

$$\frac{1}{n}\sum_{ij}(R_{ij}-\hat R_{ij})^2$$

or *mean absolute error* (MAE) 


$$\frac{1}{n}\sum_{ij}|R_{ij}-\hat R_{ij}|$$

which penalize small versus large score deviations differently.

In [ ]:
# MSE
MSE = accuracy.mse(predictions, verbose=False)
print(MSE)

In [ ]:
# MAE
MAE = accuracy.mae(predictions, verbose=False)
print(MAE)

The problem with regression metrics is that they assume a numeric score, which is often not the case with recommender systems which can be built on various user-platform interactions. A useful **ranking-based metric** that overcomes this issue is the *Fraction of Concordant Pairs* (FCP).
A *concordant pair of ratings* is composed of two pairs of true and predicted ratings $(r_{1}, \hat{r}_1)$ and $(r_{2}, \hat{r}_2)$ so that 

$$r_1 > r_2 \text{ implies }\hat r_1 > \hat r_2$$

and


$$r_2 > r_1 \text{ implies }\hat r_2 > \hat r_1.$$

That is, the predicted and true ratings agree which of Movie 1 and Movie 2 was better (no matter their actual scores). Now, FCP is the ratio of concordant rating pairs to all rating pairs. Hence, the FCP ranges between the best score 1 and worst score 0. In many cases, a metric like this captures user intent better as it discounts for people using different scales for rating movies.

In [ ]:
# FCP - Fraction of Concordant Pairs, the fraction of pairs whose relative ranking order is correct

FCP = accuracy.fcp(predictions, verbose=False)
print(FCP) 

The performance of recommender systems are often **evaluated by usage** that is, how many of the predicted items prompt the user to interaction (clicking on a recommended movie). In turn, one can use classification metrics (accuracy, precision, recall) to see what percentage of our recommendations resulted in user interaction. Such an evaluation is only possible in an online setting where we can test our system with live users but could tell us more about the actual business impact of the recommendations.


You can dive deep into various evaluation techniques and the many angles of measuring recommender system performance in [this article from Microsoft](http://www.bgu.ac.il/~shanigu/Publications/EvaluationMetrics.17.pdf).

#### (BONUS) Exercise 2

Try and increase the number of latent factors and create a test plot with the evaluation metric of your choice.

#### Solutions

We will fit the `FunkSVD` model with a varying number of factors and check the corresponding FCP score:

In [ ]:
%%time
my_train_dataset, my_test_dataset = train_test_split(my_dataset, test_size=0.2)

fcps = []
factors = range(10, 201, 5)
for n_factors in factors:    
    my_algorithm = FunkSVD(n_factors=n_factors, 
                           n_epochs=50, 
                           lr_all=0.1,    # Learning rate for each epoch
                           biased=False,  # This forces the algorithm to store all latent information in the matrices
                           verbose=0)
    
    my_algorithm.fit(my_train_dataset)
    predictions = my_algorithm.test(my_test_dataset)
    
    FCP = accuracy.fcp(predictions, verbose=False)
    fcps.append(FCP)
    print(n_factors, end="\r")

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(factors, fcps)
plt.xlabel("Latent Factor Number")
plt.ylabel("Fraction of Concordant Pairs")
plt.title("Fraction for Concordant Pairs for Various Numbers of Latent Factors")
plt.show()

We can see that there is some gain to increase the number of factors all the way up to 125 or so but there is a steep drop after 180 factors.

### Hybrid Methods

Most recommendation systems in production are actually hybrid systems: these methods combine various models when making their decision averaging scores and ranks from multiple models. These models could range from collaborative filtering and matrix factorization to deep neural networks that predict scores. This is essentially running an ensemble method and one of the most famous code competitions, the [Netflix Prize](https://en.wikipedia.org/wiki/Netflix_Prize) was won by such an ensemble too. 


<div>
<img src="https://miro.medium.com/v2/resize:fit:1000/1*rCK9VjrPgpHUvSNYw7qcuQ@2x.png" width="1000"/>
</div>

### Stale Recommendations and Time Sensitivity

If we don't update our model frequently then we're going to end up with *stale recommendations*. What we recommended last month may need to be changed for the current month: a user might have read a lot of self-help books last year but hasn't recently. 

Streaming services use the most recent user interactions, a suddenly closed movie or skipped track, to reorder their playlists in real time. Since a complete retraining of models is often not feasible computationally, there are various techniques to update model predictions with recent context for fresh recommendations. *Incremental learning* can allow updates for CF and matrix factorization models to update their parameters as new data is recorded. One can also assign weights to historical ratings which allows newer ratings to effect more the recommendations.

### Cold Start

One major issue with collaborative filtering is how do we deal with a new user with no information about them. This is also a problem with no agreed upon solution, but many heuristic based solutions. Let's say we're Netflix and we need to make recommendations to a new user:

1. We could provide them with some basic recommendations and after they view enough movies, we start to provide them with content and collaborative based recommendations.
2. We could craft some predefined classes of new users. These could be hand crafted based on our knowledge, say action movie fans, comedy fans, etc... These classes would have some recommendations associated with them. Once we have a few data points for a new user we make recommendations based on these classes and after enough data points we start making recommendations based on collaborative or content models.
3. These predefined classes for new users could be learned. We could preform unsupervised clustering of users to learn of "natural" groups. Once we have enough initial data points, we can assign the new user to their class as before and make recommendations based on this.
4. We could have users fill out a simple survey and group them into initial classes based on this.

All of these are reasonable approaches but it's hard to say which works the best, there is no hard science to this, only trying what seems reasonable.


----

<div id="container" style="position:relative;">
<div style="position:relative; float:right"><img style="height:25px""width: 50px" src ="https://drive.google.com/uc?export=view&id=14VoXUJftgptWtdNhtNYVm6cjVmEWpki1" />
</div>
</div>